In [1]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA 
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from adspy_shared_utilities import plot_labelled_scatter
from sklearn.datasets.samples_generator import make_blobs
from genetic_selection import GeneticSelectionCV
from tpot import TPOTClassifier

In [2]:
A = pd.read_csv('train.csv')
A.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [3]:
#A_test = pd.read_csv('test.csv')
#A_test.head()

In [4]:
print(A.shape)

(7352, 563)


In [5]:
print(A['Activity'].value_counts())
#print(A_test['Activity'].value_counts())

LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: Activity, dtype: int64


In [6]:
A['subject'].value_counts().head()

25    409
21    408
26    392
30    383
28    382
Name: subject, dtype: int64

In [7]:
LE = LabelEncoder()
LE.fit(A['Activity'])
Label_Activity = LE.transform(A['Activity'])
A['activity'] = Label_Activity

In [8]:
A1 = A.drop(['Activity','subject'],axis = 1)
A1.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,2
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,2
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,2
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,2
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,2


In [9]:
#A1_test = A_test.drop('Activity',axis = 1)
#A2_test = A1_test.drop('subject',axis=1)
#A2_test.head()

In [10]:
train_data_label = A1['activity'].tolist()
train_data_features = A1.drop('activity',axis = 1)
train_data_features.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [11]:
#test_data_label = A2_test['Activity'].tolist()
#test_data_features = A2_test.drop('Activity',axis = 1)
#test_data_features.head()

In [12]:
pca = PCA(100)
pca_feature = pca.fit_transform(train_data_features)
print(pca_feature.shape)


(7352, 100)


In [13]:
print(pca.components_)

[[ -7.15326815e-05  -2.99847927e-04  -2.31385358e-04 ...,  -3.60833439e-02
    2.68253762e-02   2.20742235e-02]
 [  3.25696402e-03  -4.22310863e-04  -8.39509366e-04 ...,   3.80396312e-02
   -3.83432935e-02  -1.38720372e-02]
 [ -2.45573796e-03  -6.24521614e-04   3.68280307e-04 ...,   2.89236609e-01
   -1.24780863e-01  -1.11505431e-01]
 ..., 
 [ -1.50936044e-03   1.74986638e-03   2.04147696e-02 ...,  -4.29975001e-03
   -6.04917580e-03   1.25432860e-02]
 [  1.06180504e-03  -1.29903336e-02   6.49629418e-03 ...,  -1.16608475e-02
   -2.19705494e-02   2.19416322e-02]
 [  1.31181462e-02   1.01770601e-02  -5.93712577e-02 ...,  -2.06170992e-03
   -5.36611963e-03   1.38631142e-02]]


In [14]:
#start=datetime.now()
#y= train_data_label
#X = pca_feature
#X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)
#Boost = XGBClassifier()
#HA = Boost.fit(X_train,y_train)
#Acc_train = Boost.score(X_train,y_train)
#Acc_test = Boost.score(X_test,y_test)
#print(Acc_train)
#print(Acc_test)
#print(datetime.now()-start)

In [15]:
#y_true, y_pred = test_data_label, boost.predict(test_data_features)

#prediction = Boost.predict(test_data_features)
    
#print(metrics.accuracy_score(y_true,prediction))


In [16]:
#dbscan = DBSCAN(eps = , min_samples = 1,n_jobs=-1)

In [16]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(pca_feature)
X_scaled

array([[ 0.04312048,  0.34838694,  0.13106643, ...,  0.36776812,
         0.5215941 ,  0.28915127],
       [ 0.04253741,  0.36299094,  0.0811305 , ...,  0.44351092,
         0.43360491,  0.38490183],
       [ 0.04487297,  0.38899501,  0.05332991, ...,  0.44655546,
         0.44408876,  0.3717606 ],
       ..., 
       [ 0.48335874,  0.15214831,  0.40942172, ...,  0.31797139,
         0.33831884,  0.37657273],
       [ 0.46647281,  0.12792491,  0.40942627, ...,  0.48410803,
         0.35326406,  0.38814802],
       [ 0.46944733,  0.17266903,  0.32526459, ...,  0.51235924,
         0.60228829,  0.33970996]])

In [18]:
#DB = dbscan.fit_predict(X_scaled)

In [19]:
#print(datetime.now()-start)
#dbscan.labels_

In [20]:
#for item in enumerate(dbscan.labels_):
    #if item == 10:
        #print(item,len(item))
    #elif item != 10:
        #print(0)
    

In [21]:
#X,y = make_blobs(random_state = 9, n_samples = 7352)
#plot_labelled_scatter(X,DB+1,['Cluster1','Cluster2','Cluster3','Cluster4','Cluster5' ])



In [ ]:
    start=datetime.now()
    estimator = XGBClassifier()

    selector = GeneticSelectionCV(estimator,cv=4,verbose=1,n_population=50,n_generations=2,n_jobs=-1)
    selec = selector.fit(X_scaled, train_data_label)
    print(selec.support_)
    print(datetime.now()-start)

In [ ]:
start=datetime.now()
y= train_data_label
X = X_scaled

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)
tpot = TPOTClassifier(generations = 2,population_size = 50 ,n_jobs = -1)
tpot.fit(X_train,y_train)

In [ ]:
print(tpot.score(X_train, y_train))